In [30]:
import pandas as pd
# pd.options.mode.chained_assignment = None  # default='warn'
import ast as ast
import numpy as np
from typing import *

from task_2_helpers import *
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers
from tensorflow.keras import Sequential

import nltk
from nltk.tokenize import word_tokenize

import datasets
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from textblob import TextBlob
from collections import defaultdict

import matplotlib.pyplot as plt

# from task2_business_KNN import classify_Kmeans


### Data Read

In [3]:
# df_business = pd.read_parquet('data/ATML2024_businesses.parquet')
df_business = pd.read_csv('data/ATML2024_businesses.csv', header=0)
print(df_business.shape)
# df_Train_reviews = pd.read_parquet('data/ATML2024_reviews_train.parquet')
df_Train_reviews = pd.read_csv('data/ATML2024_reviews_train.csv', header=0)

# # df_users = pd.read_parquet('data/ATML2024_users.parquet')
# df_users = pd.read_csv('data/ATML2024_users.csv', header=0, nrows=100)

(138210, 11)


In [4]:
df_business.postal_code.isna().value_counts()

postal_code
False    138145
True         65
Name: count, dtype: int64

In [5]:
zipCodes = dict(df_business.postal_code.value_counts())

val_count = 0
for z in zipCodes.keys():
    try:
        zcode = int(z)
        if zcode >= 19019 and  zcode <= 19255: ### Philadelphia zip codes
            val_count += zipCodes[str(z)]
    except:
        continue

print(val_count)

19707


In [6]:
bussiness_city = ['Phila','Philadelphia', 'Philadelphia', 'Philadelphia (Northeast Philly)', 'Philadelphia PA', 'Philadelphila', 'Philadephia', 'Philiadelphia', 
                  'Philiidelphia', 'Philly', 'Southwest Philadelphia', 'West Philadelphia', 'philadelphia', 'PHILA', 'PHILADELPHIA']
print(df_business[df_business['city'].isin(bussiness_city)].shape)


(13617, 11)


### Cleanup business data columns

In [7]:
df_business['postal_code_int'] = pd.to_numeric(df_business['postal_code'], errors='coerce', downcast='integer')
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,attributes,categories,hours,postal_code_int
0,wPYArZCy3AETdTjB-x3fpm,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",63123.0
1,QQNIWVsnAT_iKiKriWrFUt,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",85711.0
2,wm9eoqjytVbC7dQcM4WSTM,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",19107.0
3,AfVDXGIKBUE0EdTw_6cMWm,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",18054.0
4,QjvanH64QUduo6E-8F33FC,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",37015.0


In [8]:
# Convert dictionary columns to the correct types @@@@ Run only once
df_business_ph = df_business.loc[ (df_business['postal_code_int'] >= 19019) & (df_business['postal_code_int'] <= 19255) ].copy()
print(df_business_ph.shape)
df_business_ph['attributes'] = df_business_ph['attributes'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())
print(df_business_ph.shape)
df_business_ph['hours'] = df_business_ph['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) or not pd.isnull(x)  else dict())
print(df_business_ph.shape)

# df_business['weekend'] = df_business['hours'].apply(lambda x: extract_weekend_value(x))
df_business_ph['hours_list'] = df_business_ph['hours'].apply(lambda x: extract_hours_value(x))

print(df_business_ph.shape)

(19707, 12)
(19707, 12)
(19707, 12)
(19707, 13)


In [9]:
df_business_ph['attributes_list'] = df_business_ph['attributes'].apply(lambda x: sorted(extract_attributes_value(x)))
df_business_ph['categories_list'] = df_business_ph['categories'].apply(lambda x: sorted(map(lambda a: a.strip().lower(), x.split(',')) ) if isinstance(x, str) else [x])

### Join Business Data With Ratings Data and Conduct Exploratory Data Analysis (EDA)

In [10]:
df_business_data = df_business_ph[['business_id', 'postal_code_int', 'attributes_list', 'hours_list', 'categories_list']]
df_ratings_data = df_Train_reviews[['business_id', 'rating', 'useful', 'funny', 'cool', 'text']]

df_data = pd.merge(df_business_data, df_ratings_data, left_on= 'business_id', right_on='business_id')
print(df_data.shape)
df_data.head()

(176549, 10)


,business_id,postal_code_int,attributes_list,hours_list,categories_list,rating,useful,funny,cool,text
0,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,1,0,2,So I don't know how the other review was doubl...
1,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,0,I love this place.. I love their chunky fried...
2,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,1,I ordered so many cake from here for our famil...
3,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,0,0,0,Ordered a birthday cake here for the first tim...
4,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,1,0,0,This is my favorite bakery in Chinatown! It's ...


In [1]:
df_data.to_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet')

NameError: name 'df_data' is not defined

### Explanatory Data Analysis

In [3]:
df_data = pd.read_parquet('data/ATML2024_Task2_PhiliBussRatings.parquet', engine='pyarrow')
print(df_data.shape)

df_data_res = df_data[df_data['categories_list'].apply(lambda x: any('restaurants' in str(a).lower() for a in x)
                       or any('restaurant' in str(a).lower() for a in x))]
df_data_res.shape


(176549, 10)
Index(['business_id', 'postal_code_int', 'attributes_list', 'hours_list',
       'categories_list', 'rating', 'useful', 'funny', 'cool', 'text'],
      dtype='object')


(123414, 10)

In [4]:
# all_uniq_cat = set(np.concatenate(df_data.categories_list.values).ravel()) #df_data.categories_list.values.flatten()
# all_uniq_attr = set(np.concatenate(df_data.attributes_list.values).ravel()) #df_data.categories_list.values.flatten()

#### 2.0 Find out which one of the categories are related to restaurant services

Before deal with the reviews and treds in them, we need to identify which one of the businesses are actually in the restaurant and related area.

In [5]:
business_cat = np.concatenate(df_data.categories_list.values).ravel()
c = Counter(business_cat)

print(f'Total number of distinct categories of business: {len(c)}')

print(f'The most frequest categories : {c.most_common(10)}')

df_data_res = df_data[df_data['categories_list'].apply(lambda x: any('restaurants' in str(a).lower() for a in x)
                       or any('restaurant' in str(a).lower() for a in x))]
                                                                
print(df_data_res.shape)                                                    

Total number of distinct categories of business: 1081
The most frequest categories : [('restaurants', 123410), ('food', 47310), ('nightlife', 40206), ('bars', 38164), ('american (new)', 26829), ('breakfast & brunch', 22358), ('american (traditional)', 21800), ('sandwiches', 19789), ('italian', 15313), ('coffee & tea', 14008)]
(123414, 10)


### 2.2 What is the food trend of Philadelphia

In [6]:
# rating_1_cat = set(np.concatenate(df_data[df_data.rating == 1].categories_list.values).ravel())
# rating_2_cat = set(np.concatenate(df_data[df_data.rating == 2].categories_list.values).ravel())
# rating_3_cat = set(np.concatenate(df_data[df_data.rating == 3].categories_list.values).ravel())
# rating_4_cat = set(np.concatenate(df_data[df_data.rating == 4].categories_list.values).ravel())
# rating_5_cat = set(np.concatenate(df_data[df_data.rating == 5].categories_list.values).ravel())

#### 2.3 Transformer (Kyril)

In [7]:
sent_pipe = pipeline("sentiment-analysis", # or "text-classification"
                     model="distilbert-base-uncased-finetuned-sst-2-english")

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [9]:
# Use tokenize function to ignore too long 
# Define stopwords and punctuation

#stop_words = set(stopwords.words('english'))
#punctuation = set(string.punctuation)

def clean_tokens(text : str) -> List:
    
    # Tokenize the text
    token = word_tokenize(text.lower())
    
    # Remove unwanted punctuation and stop words
    # if extra_clean == True:
    #    tokens = [word for word in tokens if (word not in punctuation and word not in stop_words)]

    return token

In [10]:
df_data_res['text_tokens']  = df_data_res['text'].apply(clean_tokens)
df_data_res['tokens_count'] = df_data_res['text_tokens'].apply(len)

/tmp/ipykernel_14350/792970574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_res['text_tokens']  = df_data_res['text'].apply(clean_tokens)
/tmp/ipykernel_14350/792970574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_res['tokens_count'] = df_data_res['text_tokens'].apply(len)


In [11]:
df_data_res.to_parquet('data/ATML2024_Task2_PhiliResRatings.parquet')

Milestone 

In [2]:
df_data_res = pd.read_parquet('data/ATML2024_Task2_PhiliResRatings.parquet', engine='pyarrow')
print(df_data_res.shape)

(123414, 12)


In [13]:
df_res_short = df_data_res[df_data_res['tokens_count'] < 256]

It is possible to reduce the size of the too big reviews using a text2text pretrained model. 
Exemple bellow uses pytorch

In [14]:
"""Generate smaller text from bigger text using transformer tokenizer"""

try: 
    import torch 
    
    model_name = "allenai/led-large-16384-arxiv"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

    # apply below function to all reviews 
    def generate_smaller_text(text : str, length : int = 64) -> str:
        generated_text = pipe(
            text, 
            truncation=True, 
            max_length=length, 
            no_repeat_ngram_size=5, 
            num_beams=3, 
            early_stopping=True
        )

except ImportError as e:
    print("Error : {}".format(e))
    

Error : No module named 'torch'


In [18]:
review_list = df_res_short['text'].to_list()
# sentiments = sent_pipe(review_list)
df_sentiments = pd.DataFrame(sentiments)
df_sentiments.rename(columns={'label':'sentiment', 'score':'sent_score'}, inplace=True)

#### 2.4 Sentiment Analysis

In [3]:
# Function to calculate sentiment score from pretrained models
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

In [8]:
# Calculate sentiment for each review
df_data_res['sentiment'] = df_data_res['text'].apply(get_sentiment)

65.04774618148804


In [60]:
df_data_attr

,index,business_id,postal_code_int,attributes_list,hours_list,categories_list,rating,useful,funny,cool,text,text_tokens,tokens_count,sentiment
0,0,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,1,0,2,So I don't know how the other review was doubl...,"[so, i, do, n't, know, how, the, other, review...",182,0.177976
1,1,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,0,I love this place.. I love their chunky fried...,"[i, love, this, place, .., i, love, their, chu...",47,0.375000
2,2,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,0,0,1,I ordered so many cake from here for our famil...,"[i, ordered, so, many, cake, from, here, for, ...",35,0.366667
3,3,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",4,0,0,0,Ordered a birthday cake here for the first tim...,"[ordered, a, birthday, cake, here, for, the, f...",203,0.234841
4,4,wm9eoqjytVbC7dQcM4WSTM,19107.0,"[Alcohol, BikeParking, BusinessAcceptsCreditCa...","[[7.0, 20.0], [7.0, 20.0], [7.0, 20.0], [7.0, ...","[bakeries, bubble tea, coffee & tea, food, res...",5,1,0,0,This is my favorite bakery in Chinatown! It's ...,"[this, is, my, favorite, bakery, in, chinatown...",114,0.288961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123409,176522,gSkyrataUwvsAMli-L39n8,19104.0,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[9.0, 19.5], [9.0, 19.5], [9.0, 19.5], [9.0, ...","[american (traditional), coffee & tea, food, r...",5,4,0,1,I don't know how this place doesn't have five ...,"[i, do, n't, know, how, this, place, does, n't...",140,0.278125
123410,176545,QscEk0TPjLIlLgQzIN9TnW,19147.0,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1.0, -1.0], [11...","[mexican, restaurants]",5,2,1,1,First time trying this restaurant and I had a ...,"[first, time, trying, this, restaurant, and, i...",140,0.395739
123411,176546,QscEk0TPjLIlLgQzIN9TnW,19147.0,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1.0, -1.0], [11...","[mexican, restaurants]",5,1,0,0,Amazing food and wonderful service! We got the...,"[amazing, food, and, wonderful, service, !, we...",61,0.706399
123412,176547,QscEk0TPjLIlLgQzIN9TnW,19147.0,"[Alcohol, Ambience, BikeParking, BusinessAccep...","[[11.0, 22.0], [11.0, 22.0], [-1.0, -1.0], [11...","[mexican, restaurants]",5,1,0,0,This hidden gem in south philly is why we need...,"[this, hidden, gem, in, south, philly, is, why...",97,0.294039


In [12]:
# Initialize dictionaries to store ratings and sentiments for each attribute
attribute_ratings = defaultdict(list)
attribute_sentiments = defaultdict(list)

# Populate the dictionaries
for index, row in df_data_res.iterrows():
    for attribute in row['attributes_list']:
        attribute_ratings[attribute].append(row['rating'])
        attribute_sentiments[attribute].append(row['sentiment'])

# Calculate the mean rating and mean sentiment for each attribute
attribute_mean_ratings = {attr: sum(ratings) / len(ratings) for attr, ratings in attribute_ratings.items()}
attribute_mean_sentiments = {attr: sum(sentiments) / len(sentiments) for attr, sentiments in attribute_sentiments.items()}

In [58]:
attribute_mean_ratings

{'Alcohol': 3.785098982264783,
 'BikeParking': 3.80400225933995,
 'BusinessAcceptsCreditCards': 3.781729106628242,
 'BusinessParking': 3.78744981166439,
 'ByAppointmentOnly': 4.02927294398093,
 'Caters': 3.800903277735545,
 'OutdoorSeating': 3.7850119227172,
 'RestaurantsDelivery': 3.783843639626882,
 'RestaurantsPriceRange2': 3.7765894984063078,
 'RestaurantsTakeOut': 3.7820209391384383,
 'WiFi': 3.7988663614562896,
 'Ambience': 3.7846320067913464,
 'DogsAllowed': 3.9527969947816626,
 'GoodForKids': 3.7734096714218692,
 'GoodForMeal': 3.8226466307383657,
 'HappyHour': 3.8594646416995113,
 'HasTV': 3.7855596162240346,
 'NoiseLevel': 3.7891466409013144,
 'RestaurantsAttire': 3.7712933893433562,
 'RestaurantsGoodForGroups': 3.7733772236765937,
 'RestaurantsReservations': 3.7801114956441686,
 'RestaurantsTableService': 3.8755975333246684,
 'WheelchairAccessible': 3.896518866725956,
 'GoodForDancing': 3.7364478981368787,
 'Music': 3.714301144955714,
 'Smoking': 3.8212417557425518,
 'BYOB':